<a href="https://colab.research.google.com/github/amrangelid/imersao.ia-gemini/blob/main/Imersao_IA_Alura_e_Google_Gemini_Projeto_MentorIA_Curricular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [80]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import json

warnings.filterwarnings("ignore")

In [39]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [40]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [83]:
#############################################################################
# --- Agente 1: Analista de Currículo para processar dados estruturados --- #
#############################################################################
def processar_dados_currriculo_estruturado(nome_do_curso_input, disciplinas_input, habilidades_input):
  """
  Processa informações de currículo já fornecidas em formato estruturado.

  Args:
    nome_curso (str): O nome do curso.
    lista_disciplinas (list): Uma lista de strings, onde cada string é uma disciplina.
    lista_habilidades (list): Uma lista de strings, onde cada string é uma habilidade.

  Returns:
    dict: Um dicionário estruturado contendo as informações do currículo.
          Retorna None se o input for inválido (embora neste caso seja menos provável com input() estruturado).
  """
  print("--> Agente Analista de Currículo (dados estruturados) em execução...")

  # Verificações básicas para garantir a integridade da informação
  if not isinstance(nome_do_curso_input, str) or not isinstance(disciplinas_input, list) or not isinstance(habilidades_input, list):
      print("Erro: Input inválido para o Agente Analista de Currículo (tipos incorretos).")
      return None

  # Dicionário para armazenar as informações coletadas
  analise_currriculo_alvo = {
      "nome_do_curso": nome_do_curso_input,
      "disciplinas": disciplinas_input, # Já é uma lista, só atribuímos
      "habilidades_declaradas": habilidades_input # Já é uma lista, só atribuímos
  }

  print("--> Processamento de dados estruturados do currículo concluído.")
  return analise_currriculo_alvo # Retorna o dicionário organizado

In [84]:
#############################################
# --- Agente 2: Inteligência de Mercado --- #
#############################################
def inteligencia_mercado(data_de_hoje_input, local_input, nome_do_curso_input, analise_currriculo_alvo):

  analista_de_mercado = Agent(
      name="agente_inteligencia_mercado",
      model="gemini-2.5-flash-preview-04-17-thinking",
      description="Agente que vai analisar as necessisdade do mercado com base no currículo do usuário",
      tools=[google_search],
      # Inserir as instruções do Agente de Inteligência de Mercado #################################################
      instruction=f"""
        Você é um analista de mercado com experiência na área educação. Com base no nome do curso ({nome_do_curso_input})
        e no estado informado ({local_input}), utilize a ferramenta de busca do Google (Google Search) para pesquisar
        demandas de mercado (vagas, relatórios, artigos) na área de atuação profissional relacionada ao curso.
        Identifique as habilidades técnicas, habilidades comportamentais e tecnologias/ferramentas mais demandadas
        pelo mercado NOS ÚLTIMOS 12 MESES na região especificada. Ignore informações de cursos ou treinamentos nos
        resultados, foque em vagas e relatórios de mercado. Apresente o resultado EXCLUSIVAMENTE em formato JSON,
        usando as seguintes chaves: 'habilidades_tecnicas': Uma lista de strings com as habilidades técnicas encontradas.
        'habilidades_comportamentais': Uma lista de strings com as habilidades comportamentais encontradas.
        'tecnologias_ferramentas': Uma lista de strings com as tecnologias e ferramentas demandadas.
        'tendencias_mercado': Uma lista de strings com as tendências identificadas.
        Se não encontrar informações para alguma categoria, use uma lista vazia [].
        """
      )

  entrada_do_analista_de_mercado = f"Dia de hoje: {data_de_hoje_input}\nEstado: {local_input}\nNome do Curso: {nome_do_curso_input}\nCurrículo {analise_currriculo_alvo}"
  # Executa o agente
  inteligencia_do_mercado = call_agent(analista_de_mercado, entrada_do_analista_de_mercado)
  return inteligencia_do_mercado

In [85]:
##############################################
# --- Agente 3: Analista de Concorrência --- #
##############################################
"""
Implementação do Agente Analista de Concorrência usando o framework de agentes.

Este agente usa a ferramenta Google Search para buscar informações sobre currículos
de cursos concorrentes e tenta extrair disciplinas e habilidades em formato estruturado.
"""

def analista_concorrencia(nome_do_curso_alvo, concorrentes_opcionais=None):
    """
    Define e executa o agente para analisar currículos de cursos concorrentes.

    Args:
      nome_do_curso_alvo (str): O nome do curso que está sendo analisado (para ajudar a encontrar concorrentes similares).
      concorrentes_opcionais (list, optional): Uma lista de nomes de concorrentes específicos para buscar.
                                               Se None, o agente pode tentar encontrar concorrentes com base no nome do curso alvo.

    Returns:
      list: Uma lista de dicionários/JSON, onde cada dicionário representa a análise de um currículo concorrente.
            Retorna uma lista vazia se não encontrar informações ou se o agente não retornar no formato esperado.
    """
    print(f"--> Agente Analista de Concorrência em execução para o curso '{nome_do_curso_alvo}'...")

    # Definimos o Agente Analista de Concorrência
    analista = Agent(
        name="agente_analista_concorrencia",
        model="gemini-2.5-flash-preview-04-17-thinking",
        description="Agente que pesquisa e analisa currículos de cursos concorrentes.",
        tools=[google_search],
        # Buscar currículos e extrair informações, formatando a saída como uma LISTA de JSONs.
        instruction=f"""
          Você é um analista de currículos de cursos, especializado em identificar a estrutura e o conteúdo programático de graduações ou pós-graduações.
          Sua tarefa é pesquisar na internet (usando Google Search) por currículos, grades curriculares ou descrições de cursos que sejam concorrentes diretos ou similares ao curso "{nome_do_curso_alvo}".

          Se a lista de concorrentes opcionais for fornecida ({concorrentes_opcionais}), priorize a busca por informações desses concorrentes específicos (ex: "currículo curso X", "grade curricular universidade Y curso Z"). Se nenhuma lista for fornecida ou se a busca pelos específicos não for suficiente, faça buscas mais gerais (ex: "currículo cursos {nome_do_curso_alvo}", "comparativo cursos {nome_do_curso_alvo}").

          Para CADA currículo concorrente encontrado (busque por pelo menos 2 ou 3 se possível, mas retorne o que encontrar), extraia as seguintes informações:
          - Nome do curso concorrente.
          - Uma lista das principais disciplinas ou módulos.
          - Uma lista das habilidades ou focos que o currículo parece desenvolver ou declarar.

          Apresente o resultado EXCLUSIVAMENTE como uma LISTA de objetos JSON. Cada objeto JSON na lista deve representar um currículo concorrente analisado e deve ter as seguintes chaves:
          'nome_do_curso_concorrente': (string) O nome do curso concorrente encontrado.
          'disciplinas': (lista de strings) Uma lista das principais disciplinas ou módulos extraídos.
          'habilidades_declaradas_concorrente': (lista de strings) Uma lista de habilidades ou focos identificados no currículo concorrente.
          'fonte_url_simulada': (string) A URL do currículo encontrado (simulada ou real se a ferramenta a fornecer).

          Se não encontrar informações suficientes para um concorrente, inclua um objeto JSON básico com o nome e listas vazias. Se não encontrar nenhum concorrente relevante, retorne uma lista JSON vazia [].
          """
    )

    entrada_para_analista = f"Curso Alvo: {nome_do_curso_alvo}\nConcorrentes Opcionais: {concorrentes_opcionais if concorrentes_opcionais else 'Nenhum fornecido. Faça buscas gerais.'}"
    # Executa o agente
    # A saída será o texto gerado pelo modelo, que esperamos que seja uma string JSON representando uma lista.
    resultado_texto_agente = call_agent(analista, entrada_para_analista)

    analise_da_concorrencia = []
    try:
        # Carregar a string como JSON
        analise_da_concorrencia = json.loads(resultado_texto_agente)
        print("--> Saída do Agente 3 convertida para lista JSON com sucesso.")

        # Verificação básica para garantir que é uma lista
        if not isinstance(analise_da_concorrencia, list):
             print("--> Aviso: Saída do Agente 3 não é uma lista, mesmo após parse JSON.")
             analise_da_concorrencia = [] # Reseta para lista vazia se não for o formato esperado

    except json.JSONDecodeError:
        # Se houver um erro ao tentar converter (porque a saída não era um JSON válido)
        print(f"--> Erro: Saída do Agente 3 não é um JSON válido. Saída: {resultado_texto_agente}")
        analise_da_concorrencia = [] # Retorna uma lista vazia em caso de erro


    print("--> Agente Analista de Concorrência finalizado.")
    # Retornamos a lista de dicionários com a análise dos concorrentes (ou lista vazia se houver erro).
    return analise_da_concorrencia

In [86]:
##########################################################
# --- Agente 4: Mapeamento e Diagnóstico Comparativo --- #
##########################################################
def mapeamento_diagnostico_comparativo(currriculo_alvo, inteligencia_mercado, analise_concorrencia):
    """
    Realiza o mapeamento e diagnóstico comparativo entre currículo, mercado e concorrência.

    Args:
      currriculo_alvo (dict): Resultado da análise do currículo alvo (do Agente 1).
      inteligencia_mercado (dict): Resultado da inteligência de mercado (do Agente 2).
      analise_concorrencia (list): Resultado da análise de concorrentes (do Agente 3).

    Returns:
      dict: Um dicionário contendo os resultados das análises comparativas (GAPs, pontos fortes, etc.).
    """
    print("--> Agente de Mapeamento e Diagnóstico Comparativo (Agente 4) em execução...")

    # Dicionário para armazenar todos os resultados da análise comparativa
    resultados_comparativos = {}

    # --- Análise: Currículo Alvo vs. Demandas do Mercado (Identificação de GAPs) ---
    print("--> Analisando GAPs do currículo alvo em relação às demandas do mercado...")

    # Habilidades declaradas do currículo alvo (Agente 1)
    habilidades_curriculo = currriculo_alvo.get("habilidades_declaradas", [])
    # Demandas de habilidades técnicas e tecnologias do mercado (Agente 2)
    demandas_tecnicas_mercado = inteligencia_mercado.get("habilidades_tecnicas_demandadas", [])
    demandas_tecnologias_mercado = inteligencia_mercado.get("tecnologias_ferramentas_demandadas", [])

    # Juntamos todas as demandas relevantes do mercado para comparar com o currículo
    todas_demandas_mercado = demandas_tecnicas_mercado + demandas_tecnologias_mercado

    # Lista para guardar os GAPs encontrados
    gaps_encontrados = []

    # Lógica para encontrar GAPs:
    # Percorremos cada item das demandas do mercado...
    for demanda in todas_demandas_mercado:
        # Se essa demanda NÃO está na lista de habilidades declaradas do currículo.
        # Usamos .lower() e .strip() para fazer uma comparação que ignore maiúsculas/minúsculas e espaços extras.
        demanda_limpa = demanda.lower().strip()
        encontrado_no_curriculo = False
        for habilidade in habilidades_curriculo:
            if habilidade.lower().strip() == demanda_limpa:
                encontrado_no_curriculo = True
                break # Encontrou, não precisa verificar o resto das habilidades do currículo

        # Se a demanda não foi encontrada em nenhuma das habilidades declaradas no currículo...
        if not encontrado_no_curriculo:
            # ... e se ainda não adicionamos essa demanda à nossa lista de GAPs...
            if demanda not in gaps_encontrados:
                # ... então, é um GAP! Adicionamos à lista.
                gaps_encontrados.append(demanda)

    # Armazenamos os GAPs encontrados nos resultados comparativos
    resultados_comparativos["gaps_mercado_curriculo"] = gaps_encontrados

    print(f"--> Identificação de GAPs concluída. GAPs encontrados: {gaps_encontrados}")

    # --- Análise: Currículo Alvo vs. Demandas do Mercado (Identificação de Pontos Fortes) ---
    # Pontos fortes seriam habilidades do currículo que são demandadas pelo mercado.
    print("--> Analisando Pontos Fortes do currículo alvo em relação às demandas do mercado...")
    pontos_fortes_encontrados = []

    # Percorremos cada habilidade declarada no currículo...
    for habilidade_curriculo in habilidades_curriculo:
        # ... e verificamos se essa habilidade está nas demandas do mercado.
        habilidade_curriculo_limpa = habilidade_curriculo.lower().strip()
        encontrado_no_mercado = False
        for demanda in todas_demandas_mercado:
            if demanda.lower().strip() == habilidade_curriculo_limpa:
                encontrado_no_mercado = True
                break # Encontrou no mercado

        # Se a habilidade do currículo foi encontrada nas demandas do mercado...
        if encontrado_no_mercado:
             # ... e se ainda não adicionamos essa habilidade à nossa lista de pontos fortes...
             if habilidade_curriculo not in pontos_fortes_encontrados:
                 # ... então, é um ponto forte!
                 pontos_fortes_encontrados.append(habilidade_curriculo)

    # Armazenamos os pontos fortes encontrados
    resultados_comparativos["pontos_fortes_mercado_curriculo"] = pontos_fortes_encontrados

    print(f"--> Identificação de Pontos Fortes concluída. Pontos Fortes encontrados: {pontos_fortes_encontrados}")

    # --- Análise: Currículo Alvo vs. Concorrência ---
    # Esta parte pode ser mais complexa, pois envolve comparar listas de disciplinas/habilidades
    # entre múltiplos concorrentes e o currículo alvo. Ficará para desenvolvimento futuro
    print("--> Análise comparativa com concorrentes (A SER IMPLEMENTADA).")
    # Exemplo de como você poderia armazenar resultados futuros:
    # resultados_comparativos["comparativo_concorrencia"] = {
    #    "diferenciais_alvo": [],
    #    "desvantagens_alvo": []
    # }

    print("--> Agente de Mapeamento e Diagnóstico Comparativo finalizado.")
    # Retornamos o dicionário com os resultados da análise (por enquanto, GAPs e Pontos Fortes vs Mercado).
    return resultados_comparativos

In [89]:
##########################################################
# --- Agente 5: Gerador de Relatório e Recomendações --- #
##########################################################

def gerar_relatorio_recomendacoes(resultados_analise_comparativa):
    """
    Define e executa o agente para gerar o relatório final e recomendações.

    Args:
      resultados_analise_comparativa (dict): O dicionário de resultados da análise comparativa (do Agente 5),
                                              contendo GAPs, pontos fortes, etc.

    Returns:
      str: O texto formatado do relatório gerado pelo agente.
           Retorna uma string indicando erro se algo falhar.
    """
    print("--> Agente Gerador de Relatório e Recomendações (Agente 5) em execução...")

    # Definimos o Agente Gerador de Relatório
    gerador_relatorio = Agent(
        name="agente_gerador_relatorio",
        model="gemini-2.5-flash-preview-04-17-thinking",
        description="Agente que gera um relatório analítico e recomendações para currículos de cursos.",
        instruction=f"""
          Você é um consultor especializado em currículos educacionais e alinhamento com o mercado de trabalho.
          Você recebeu os resultados de uma análise comparativa do currículo de um curso específico.
          Os resultados da análise estão formatados em um dicionário Python/JSON, contendo informações como GAPs em relação às demandas do mercado, pontos fortes do currículo e (futuramente) uma comparação com concorrentes.

          Seu objetivo é gerar um relatório CLARO, OBJETIVO e ACIONÁVEL para o usuário.
          O relatório deve:

          1.  Começar com um título apropriado (ex: "Relatório de Análise Curricular e Alinhamento de Mercado").
          2.  Ter uma introdução breve explicando o propósito do relatório (analisar o currículo em relação ao mercado e concorrência).
          3.  Apresentar a seção "Análise de Alinhamento com o Mercado":
              - Descrever os principais GAPs encontrados (habilidades/tecnologias que o mercado demanda mas que não estão no currículo alvo). Liste-os de forma clara.
              - Descrever os Pontos Fortes identificados (habilidades do currículo alvo que são demandadas pelo mercado). Liste-os.
              - (Opcional, se dados disponíveis) Comentar sobre Excedentes (habilidades no currículo com baixa demanda no mercado).
          4.  (FUTURO) Ter uma seção "Análise Comparativa com Concorrentes" (quando essa lógica for implementada no Agente 5):
              - Destacar os diferenciais do currículo alvo em relação aos concorrentes.
              - Apontar áreas onde os concorrentes parecem mais fortes ou oferecem algo a mais.
          5.  Ter uma seção "Recomendações para Aprimoramento":
              - Com base nos GAPs identificados, sugerir RECOMENDAÇÕES CONCRETAS para o aprimoramento do currículo. Exemplos: sugerir inclusão de tópicos/disciplinas, dar mais ênfase a certas áreas, considerar ferramentas/tecnologias específicas. Baseie as recomendações principalmente nos GAPs identificados.
              - (Opcional, se dados disponíveis) Sugerir como capitalizar os pontos fortes.
          6.  Ter uma conclusão breve.

          Formate o relatório usando Markdown para facilitar a leitura (títulos, subtítulos, listas). Mantenha um tom profissional, mas acessível.

          Os dados da análise comparativa para gerar o relatório são os seguintes (formato dicionário/JSON):
          {resultados_analise_comparativa}
          """
    )

    entrada_para_gerador = f"Dados da Análise: {resultados_analise_comparativa}"

    # Executa o agente. A saída será o texto do relatório.
    try:
        relatorio_final_texto = call_agent(gerador_relatorio, entrada_para_gerador)
        print("--> Agente 5 gerou o relatório com sucesso.")
        return relatorio_final_texto # Retorna a string do relatório

    except Exception as e:
        print(f"--> Erro ao executar o Agente 5: {e}")
        return "Erro ao gerar o relatório."

In [92]:
###############################
# --- Agente Orquestrador --- #
###############################

# --- ETAPA 1: ENTRADA DO USUÁRIO ---
print("--- Etapa 1: Coletando informações do usuário ---")
# Você pode usar o seu código de input() aqui para coletar os dados reais do usuário.
# Por enquanto, vamos usar valores de exemplo para poder rodar o fluxo completo.
nome_do_curso_input = input("🎓 Entre com o nome do curso alvo: \n")
print("\n")
# Coleta de disciplinas (usando o loop while que você criou antes)
disciplinas_input = []
while True:
    linha = input("📚 Digite uma disciplina (ou Enter para terminar): ")
    if linha == "": break
    disciplinas_input.append(linha)
print("\n")
# Coleta de habilidades (usando o loop while que você criou antes)
habilidades_input = []
while True:
    linha = input("🧩 Digite uma habilidade (ou Enter para terminar): ")
    if linha == "": break
    habilidades_input.append(linha)
print("\n")

local_input = input("📍 Em qual estado do Brasil você quer analisar o mercado? (Ex: São Paulo)\n")
print("\n")

concorrentes_input_str = input("🏆 Nomes de concorrentes específicos para pesquisar (separados por vírgula), ou Enter para busca geral: \n")
concorrentes_input = [c.strip() for c in concorrentes_input_str.split(',') if c.strip()] if concorrentes_input_str else None # Cria uma lista ou None

data_de_hoje_input = date.today().strftime("%d de %B de %Y") # Pega a data de hoje formatada

print("\nInformações iniciais coletadas.")
print("-" * 30)

# --- ETAPA 2: Agente 1 - Analista de Currículo Alvo ---
print("--- Etapa 2: Analista de Currículo Alvo ---")
# Chama a função que organiza os dados do input em um dicionário.
resultado_analise_currriculo_alvo = processar_dados_currriculo_estruturado(
    nome_do_curso_input,
    disciplinas_input,
    habilidades_input
)
print("Resultado do Agente 1:", resultado_analise_currriculo_alvo)
print("-" * 30)

# --- ETAPA 3: Agente 2 - Inteligência de Mercado ---
print("--- Etapa 3: Inteligência de Mercado ---")
# Chama a função do Agente 2. Ela usará o modelo e ferramentas para buscar e processar.
resultado_inteligencia_mercado = inteligencia_mercado(
    data_de_hoje_input,
    local_input,
    nome_do_curso_input,
    resultado_analise_currriculo_alvo # Passa o resultado do Agente 1 como contexto
)
# O Agente 2 deve retornar uma STRING JSON. Precisamos converter para dicionário Python.
try:
    analise_mercado_dict = json.loads(resultado_inteligencia_mercado)
    print("Resultado do Agente 2 (convertido para dict):", analise_mercado_dict)
except json.JSONDecodeError:
    print(f"ERRO: Agente 2 não retornou JSON válido. Saída: {resultado_inteligencia_mercado}")
    analise_mercado_dict = {} # Usa um dicionário vazio em caso de erro

print("-" * 30)

# --- ETAPA 4: Agente 3 - Analista de Concorrência ---
print("--- Etapa 4: Analista de Concorrência ---")
# Resultado do Agente 3 já é a lista
analise_concorrencia_lista = analista_concorrencia( # <-- Atribui o resultado (que já é lista) diretamente
    nome_do_curso_input, # Nome do curso alvo
    concorrentes_input  # Lista de concorrentes opcionais
)
print("Resultado do Agente 3:", analise_concorrencia_lista)

print("-" * 30)

# --- ETAPA 5: Agente 4 - Mapeamento e Diagnóstico Comparativo ---
print("--- Etapa 5: Mapeamento e Diagnóstico Comparativo ---")
# Chama a função do Agente 4 com os resultados dos agentes anteriores.
resultado_analise_comparativa = mapeamento_diagnostico_comparativo(
    resultado_analise_currriculo_alvo, # Output do Agente 1 (já é dict)
    analise_mercado_dict,              # Output do Agente 2 (convertido para dict)
    analise_concorrencia_lista         # Output do Agente 3 (convertido para lista)
)
print("Resultado do Agente 4:", resultado_analise_comparativa)
print("-" * 30)

# --- ETAPA 6: Agente 5 - Gerador de Relatório e Recomendações ---
print("--- Etapa 6: Gerador de Relatório e Recomendações ---")
# Chama a função do Agente 5 com o resultado da análise comparativa.
relatorio_final = gerar_relatorio_recomendacoes(resultado_analise_comparativa)

print("-" * 30) # Linha antes do relatório
# --- ETAPA 7: Output Final ---
print("--- Etapa 7: Relatório Analítico Final ---")
print(relatorio_final) # Imprime o relatório gerado pelo Agente 6
print("-" * 30)

print("\n--- Fluxo do Agente Orquestrador CONCLUÍDO ---")

# Nota Final: Para rodar este script completo, você precisa ter as definições
# de todas as funções dos agentes (processar_dados_currriculo_estruturado,
# inteligencia_mercado, analista_concorrencia, mapeamento_diagnostico_comparativo,
# gerar_relatorio_recomendacoes) e as classes/funções do framework de agentes
# (Agent, call_agent) e ferramentas (Google Search) disponíveis no seu ambiente.

--- Etapa 1: Coletando informações do usuário ---
🎓 Entre com o nome do curso alvo: 
Ciência de Dados e Inteligência Artificial


📚 Digite uma disciplina (ou Enter para terminar): Lógica e Algoritmos
📚 Digite uma disciplina (ou Enter para terminar): Introdução à Programação com Python
📚 Digite uma disciplina (ou Enter para terminar): Fundamentos de Matemática para Computação
📚 Digite uma disciplina (ou Enter para terminar): Estatística Descritiva e Probabilidade
📚 Digite uma disciplina (ou Enter para terminar): Pensamento Computacional
📚 Digite uma disciplina (ou Enter para terminar): Projeto Integrador I: Dados no dia a dia
📚 Digite uma disciplina (ou Enter para terminar): Estrutura de Dados e Algoritmos
📚 Digite uma disciplina (ou Enter para terminar): Banco de Dados Relacional (SQL)
📚 Digite uma disciplina (ou Enter para terminar): Cálculo Diferencial e Integral
📚 Digite uma disciplina (ou Enter para terminar): Visualização de Dados e Storytelling
📚 Digite uma disciplina (ou Enter p